In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s44.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
44-10938,0,0,8,8,8,99.0,73,40.0,77,117.0,...,0,0,0,0,0,0,25,20,1,0
44-01887,0,0,8,8,8,99.0,78,0.0,77,154.0,...,0,0,0,0,0,0,34,24,1,1
44-13761,0,0,8,8,8,99.0,87,0.0,77,148.0,...,0,0,0,0,0,0,26,31,1,1
44-02977,0,0,8,8,8,0.0,74,50.0,77,124.4,...,0,0,0,0,0,0,27,40,1,1
44-10546,0,0,8,8,8,1.8,71,90.0,77,117.0,...,0,0,0,0,1,0,34,24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44-19833,0,0,8,8,8,1.0,74,70.0,77,124.4,...,0,0,0,0,0,0,19,31,0,1
44-07454,0,0,8,8,8,99.0,98,20.0,77,158.0,...,0,0,0,0,0,0,36,24,1,1
44-04903,0,0,8,8,8,2.9,69,60.0,77,145.0,...,0,1,0,0,0,0,27,24,1,1


In [3]:
X.shape

(16014, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(11209, 195)
(16014, 195)
(16014,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(1320, 195)
(1320,)
[660 660]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[3069 1453]
 [  27  256]]
              precision    recall  f1-score   support

           0       0.99      0.68      0.81      4522
           1       0.15      0.90      0.26       283

    accuracy                           0.69      4805
   macro avg       0.57      0.79      0.53      4805
weighted avg       0.94      0.69      0.77      4805

ROC AUC prob1: 0.8360922572488172
MCC: 0.28685394522940977
F2: 0.4505455825413586
PR_AUC: 0.2806320711284951
CPU times: user 522 ms, sys: 6.69 ms, total

In [12]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[3141 1381]
 [  36  247]]
              precision    recall  f1-score   support

           0       0.99      0.69      0.82      4522
           1       0.15      0.87      0.26       283

    accuracy                           0.71      4805
   macro avg       0.57      0.78      0.54      4805
weighted avg       0.94      0.71      0.78      4805

ROC AUC prob1: 0.8379700029537573
MCC: 0.2822335975116122
F2: 0.447463768115942
PR_AUC: 0.25476426452093576
CPU times: user 3.68 s, sys: 5.22 ms, total:

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 14:34:05,490] A new study created in memory with name: no-name-a8a26de2-2394-49fe-99e6-0f9cf989248e
[I 2021-12-10 14:34:06,908] Trial 0 finished with value: 0.2829188039431525 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.2829188039431525.


MCC: [0.283405297709138, 0.2954640751594321, 0.2644634897779625, 0.28834235312607726]
ROC: [0.8456032512864803, 0.8416133632012667, 0.8369999505068548, 0.8322796675249583]
PR_AUC: [0.25538554986480816, 0.23259926735107828, 0.2706580759149826, 0.23000282333646943]


[I 2021-12-10 14:34:10,421] Trial 1 finished with value: 0.2802069120851233 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.2829188039431525.


MCC: [0.27581341604885756, 0.2862248341445119, 0.269153359269899, 0.28963603887722494]
ROC: [0.8403842347691534, 0.8371792795710534, 0.8229776425965004, 0.8351741428377828]
PR_AUC: [0.245031251816038, 0.2297979488852732, 0.2166896578982507, 0.2337646712027493]


[I 2021-12-10 14:34:13,641] Trial 2 finished with value: 0.28311784656096983 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 2 with value: 0.28311784656096983.


...

MCC: [0.28261121775873205, 0.29232405764940406, 0.27959309680988476, 0.3002467673012711]
ROC: [0.848924596962827, 0.851574588866098, 0.8437591393591988, 0.8496764466729909]
PR_AUC: [0.30430704248314816, 0.253521147015179, 0.28215343091883727, 0.26965358743123874]


[I 2021-12-10 14:35:38,445] Trial 23 finished with value: 0.2862917039477315 and parameters: {'n_estimators': 85, 'subsample': 0.8302098329306508, 'max_depth': 20, 'max_features': 16, 'learning_rate': 0.08216653449054163, 'min_samples_leaf': 9}. Best is trial 9 with value: 0.29408360597713357.


MCC: [0.289383340702955, 0.30126403698310233, 0.2585623466721122, 0.29595709143275645]
ROC: [0.8464230450897836, 0.8488430674007702, 0.8280878098383374, 0.8402346749785428]
PR_AUC: [0.23779705274223092, 0.24800059805286528, 0.21146035211602657, 0.21596214632190155]


[I 2021-12-10 14:35:41,556] Trial 24 finished with value: 0.2898508668988375 and parameters: {'n_estimators': 99, 'subsample': 0.8196009430772153, 'max_depth': 5, 'max_features': 18, 'learning_rate': 0.08943759311949451, 'min_samples_leaf': 13}. Best is trial 9 with value: 0.29408360597713357.


MCC: [0.2790643825474787, 0.2986425600552829, 0.2845868717265862, 0.29710965326600225]
ROC: [0.8498911439778329, 0.853379484688186, 0.8347761335055094, 0.8488819623255184]
PR_AUC: [0.2762514426655979, 0.2682334977424441, 0.2547028211952381, 0.25552116547488896]
CPU times: user 1min 35s, sys: 595 ms, total: 1min 36s
Wall time: 1min 36s


In [14]:
best = study.best_trial
best

FrozenTrial(number=9, values=[0.29408360597713357], datetime_start=datetime.datetime(2021, 12, 10, 14, 34, 47, 480465), datetime_complete=datetime.datetime(2021, 12, 10, 14, 34, 49, 586874), params={'n_estimators': 89, 'subsample': 0.8002853761125517, 'max_depth': 4, 'max_features': 17, 'learning_rate': 0.14292147161182367, 'min_samples_leaf': 35}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=9, state=TrialState.COMPLETE, value=None)

In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[3150 1372]
 [  38  245]]
              precision    recall  f1-score   support

           0       0.99      0.70      0.82      4522
           1       0.15      0.87      0.26       283

    accuracy                           0.71      4805
   macro avg       0.57      0.78      0.54      4805
weighted avg       0.94      0.71      0.78      4805

ROC AUC prob1: 0.8442776031744295
MCC: 0.28017342603227563
F2: 0.44561658785012737
PR_AUC: 0.26059185559157483
CPU times: user 197 ms, sys: 3.01 ms, total: 200 ms
Wall time: 197 ms


In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[3259 1263]
 [  44  239]]
              precision    recall  f1-score   support

           0       0.99      0.72      0.83      4522
           1       0.16      0.84      0.27       283

    accuracy                           0.73      4805
   macro avg       0.57      0.78      0.55      4805
weighted avg       0.94      0.73      0.80      4805

ROC AUC prob1: 0.8441115520041009
MCC: 0.2870702102708204
F2: 0.45368261199696286
PR_AUC: 0.2692526257562669
CPU times: user 868 ms, sys: 6.01 ms, total: 874 ms
Wall time: 871 ms


In [17]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delmode            0.261279
TrialLabor         0.086231
CS_FTP             0.060727
transfus_yes       0.058034
Episiotomy         0.044051
Lac_Min            0.043800
Admefface          0.040804
Dilat_lst          0.030375
new_age            0.021443
BESTGA             0.020973
new_BMI            0.020449
Antefetdistress    0.018872
AdmSBP             0.016804
AdmDBP             0.014495
Admreason          0.014314
chorio9            0.013997
Admcontract        0.010723
prelaborCD         0.010661
uscar              0.010136
HxnumCS            0.010124
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode             0.135576
Admefface           0.100974
transfus_yes        0.080779
CS_FTP              0.077234
Episiotomy          0.049818
spontlabor          0.043917
Lac_Min             0.039844
Dilat_lst           0.028894
Hxcsection          0.025010
new_age             0.024443
prelaborCD          0.021669
AdmSBP              0.020131
BESTGA              0.019069
TrialLabor          0.018761
vertex              0.018372
Intrafetdistress    0.017526
new_BMI             0.016969
AdmDBP              0.015993
uscar               0.015188
momrace_new         0.014002
dtype: float64

In [21]:
gb_params # 5 runs

{'n_estimators': 89,
 'subsample': 0.8002853761125517,
 'max_depth': 4,
 'max_features': 17,
 'learning_rate': 0.14292147161182367,
 'min_samples_leaf': 35,
 'random_state': 42}

In [2]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [3]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [24]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>